In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, confusion_matrix
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings

import wandb

# Set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [2]:
# --- WandB login ---
try:
    with open("wandb_api_key.txt", "r") as file:
        wandb_api_key = file.read().strip()
    wandb.login(key=wandb_api_key)
    print("WandB login successful using wandb_api_key.")
except Exception as e:
    print(f"Failed to login to WandB: {e}. Falling back to manual login.")
    wandb.login()

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Nekloyh\_netrc
wandb: Currently logged in as: nekloyh (nekloyh-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


WandB login successful using wandb_api_key.


In [3]:
# --- Configuration Section ---
class DataConfig:
    """Configuration for audio data processing parameters."""
    # General audio processing settings
    SEED = 42  # Random seed for reproducibility

    SR = 16000  # Sample rate (Hz)
    N_FFT = 2048  # FFT window size
    HOP_LENGTH = 512  # Hop length for spectrogram
    N_MELS = 128  # Number of Mel bands
    FMIN = 0.0  # Minimum frequency (Hz)
    FMAX = 8000.0  # Maximum frequency (Hz)

    # Augmentation settings
    NUM_TIME_MASKS = 2  # Number of time masks for SpecAugment
    NUM_FREQ_MASKS = 2  # Number of frequency masks for SpecAugment
    TIME_MASK_MAX_WIDTH = 60  # Maximum width of time mask
    FREQ_MASK_MAX_WIDTH = 25  # Maximum width of frequency mask
    MASK_REPLACEMENT_VALUE = -80.0  # Value for masked regions in spectrogram
    NORM_EPSILON = 1e-6  # Small value to prevent division by zero
    LOUDNESS_LUFS = -23.0  # Target loudness (LUFS)

    # Dataset and processing options
    USE_GLOBAL_NORMALIZATION = False  # Use global mean/std for normalization
    USE_RANDOM_CROPPING = True  # Apply random cropping to spectrograms
    CACHE_DIR = "F:\\Deepfake-Audio-Detector\\processed_dataset" # Directory for processed data


In [4]:
class ModelConfig:
    """Configuration for model-specific dataset creation."""

    def __init__(
        self,
        name: str,
        audio_length_seconds: float,
        overlap_ratio: float,
        apply_augmentation: bool = False,
        apply_waveform_augmentation: bool = False,
    ):
        self.name = name
        self.audio_length_seconds = audio_length_seconds
        self.overlap_ratio = overlap_ratio
        self.apply_augmentation = apply_augmentation
        self.apply_waveform_augmentation = apply_waveform_augmentation
        # Calculate max_frames_spec
        frames = (audio_length_seconds * DataConfig.SR) / DataConfig.HOP_LENGTH
        self.max_frames_spec = int(np.ceil(frames))


In [ ]:
ALL_MODEL_CONFIGS = {
    "cnn_balanced_dataset": ModelConfig(
        name="cnn_balanced_dataset",
        audio_length_seconds=8.192,
        overlap_ratio=0.5,
        apply_augmentation=True,
        apply_waveform_augmentation=True,
    ),
    "cnn_performance_dataset": ModelConfig(
        name="cnn_performance_dataset",
        audio_length_seconds=4.096,
        overlap_ratio=0.75,
        apply_augmentation=True,
        apply_waveform_augmentation=True,
    ),
}


In [6]:
# --- 1. CNN Configuration ---
class CNNConfig:
    def __init__(
        self,
        name: str,
        image_size: tuple,
        channels: int = 1,
        num_classes: int = 2,
        blocks: list = [3, 4, 6, 3],
        filters: list = [64, 128, 256, 512],
        bottleneck: bool = False,
    ):
        self.name = name
        self.image_height, self.image_width = image_size
        self.channels = channels
        self.num_classes = num_classes
        self.blocks = blocks
        self.filters = filters
        self.bottleneck = bottleneck

        assert len(blocks) == len(filters), "Number of blocks must match number of filter stages."
        assert all(f > 0 for f in filters), "All filter sizes must be positive."
        assert all(b >= 0 for b in blocks), "All block counts must be non-negative."


In [7]:
# --- 2. CNN Architecture (ResNet-like) ---
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class BottleneckBlock(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.downsample is not None:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class AudioCNN(nn.Module):
    def __init__(self, config: CNNConfig):
        super().__init__()
        self.config = config
        block = BottleneckBlock if config.bottleneck else BasicBlock
        self.in_channels = 64

        self.conv1 = nn.Conv2d(config.channels, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        layers = []
        for num_blocks, out_channels in zip(config.blocks, config.filters):
            layers.append(self._make_layer(block, out_channels, num_blocks))
        self.layers = nn.Sequential(*layers)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(config.filters[-1] * block.expansion, config.num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layers(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [8]:
# --- 3. Dataset Class for Cached Data ---
class AudioDataset(Dataset):
    def __init__(self, cache_dir: str, set_type: str, n_mels: int, max_frames_spec: int):
        self.cache_path = os.path.join(cache_dir, set_type)
        self.metadata_path = os.path.join(self.cache_path, "metadata.csv")
        self.n_mels = n_mels
        self.max_frames_spec = max_frames_spec

        if not os.path.exists(self.metadata_path):
            raise FileNotFoundError(
                f"Metadata file not found: {self.metadata_path}. Please run data_preprocessing.py first."
            )

        self.metadata = pd.read_csv(self.metadata_path)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        npy_path = os.path.join(self.cache_path, row["npy_path"])
        label = row["label"]

        try:
            spectrogram = np.load(npy_path)
            # Ensure spectrogram is 3D (channels, height, width) for CNN
            if spectrogram.ndim == 2:
                spectrogram = np.expand_dims(spectrogram, axis=0)
            elif spectrogram.ndim == 3 and spectrogram.shape[0] != 1:
                raise ValueError(
                    f"Unexpected spectrogram shape: {spectrogram.shape}. Expected (1, N_MELS, N_FRAMES)."
                )

            # Convert to float32 and then to tensor
            spectrogram = torch.from_numpy(spectrogram).float()

        except Exception as e:
            warnings.warn(f"Error loading or processing {npy_path}: {e}")
            return None  # Return None to be filtered by collate_fn

        return spectrogram, torch.tensor(label).long()

# Custom collate function to filter None values
def custom_collate_fn(batch):
    batch = [item for item in batch if item is not None]
    if not batch:
        return None
    return torch.utils.data.dataloader.default_collate(batch)


In [9]:
# --- 4. Training and Evaluation Functions ---
def train_model(
    model,
    train_loader,
    val_loader,
    optimizer,
    criterion,
    device,
    num_epochs,
    run_name,
    dataset_name,
):
    model.to(device)
    best_val_f1 = -1

    # Initialize wandb run
    wandb.init(
        project="audio-deepfake-detection",
        name=run_name,
        config={
            "learning_rate": optimizer.defaults["lr"],
            "epochs": num_epochs,
            "batch_size": train_loader.batch_size,
            "model_name": model.__class__.__name__,
            "model_config": model.config.__dict__,
            "dataset_name": dataset_name,
            "device": str(device),
        },
    )
    wandb.watch(model, log_freq=100)  # Log gradients and model parameters

    print(f"Starting training for {num_epochs} epochs on {device}...")

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        pbar = tqdm(
            train_loader, desc=f"Epoch {epoch + 1}/{num_epochs} [Train]", leave=False
        )
        for batch_idx, batch in enumerate(pbar):
            if batch is None:  # Skip empty batches from collate_fn
                continue
            data, labels = batch
            if -1 in labels.cpu().numpy():  # Check if any -1 label exists
                continue

            data, labels = data.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=loss.item())

        avg_train_loss = total_loss / len(train_loader)

        # Validation
        val_loss, val_preds, val_labels, val_probs = evaluate_model(
            model, val_loader, criterion, device
        )
        val_acc = accuracy_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds, average="binary")
        val_roc_auc = roc_auc_score(val_labels, val_probs[:, 1])

        print(
            f"Epoch {epoch + 1} Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}"
        )

        # Log metrics to wandb
        wandb.log(
            {
                "epoch": epoch,
                "train_loss": avg_train_loss,
                "val_loss": val_loss,
                "val_accuracy": val_acc,
                "val_f1_score": val_f1,
                "val_roc_auc": val_roc_auc,
            }
        )

        # Save best model
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            model_save_path = f"best_cnn_model_{run_name}.pth"
            torch.save(model.state_dict(), model_save_path)
            print(f"Saved best model with F1: {best_val_f1:.4f} to {model_save_path}")
            wandb.save(model_save_path)  # Save model to wandb

    return model


def evaluate_model(model, data_loader, criterion, device, return_cm=False):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    all_probs = []

    with torch.no_grad():
        pbar = tqdm(data_loader, desc="Evaluation", leave=False)
        for batch in pbar:
            if batch is None:  # Skip empty batches from collate_fn
                continue
            data, labels = batch
            if -1 in labels.cpu().numpy():  # Check if any -1 label exists
                continue

            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            probs = torch.softmax(outputs, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())

    avg_loss = total_loss / len(data_loader)

    if return_cm:
        cm = confusion_matrix(all_labels, all_preds)
        return (
            avg_loss,
            np.array(all_preds),
            np.array(all_labels),
            np.array(all_probs),
            cm,
        )
    else:
        return avg_loss, np.array(all_preds), np.array(all_labels), np.array(all_probs)


def plot_confusion_matrix(cm, labels=["Real", "Fake"], run_name="", save_dir="."):
    plt.figure(figsize=(8, 6))
    sns.heatmap(
        cm, annot=True, fmt="d", cmap="Blues", xticklabels=labels, yticklabels=labels
    )
    plt.title(f"Confusion Matrix for {run_name}")
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, f"confusion_matrix_{run_name}.png")
    plt.savefig(save_path)
    print(f"Confusion matrix saved to {save_path}")
    plt.show()
    return save_path


In [10]:
# --- Training Configuration ---
class TrainingConfig:
    """Configuration for training parameters."""

    def __init__(
        self,
        model_size: str,
        dataset_name: str,
        epochs: int,
        learning_rate: float,
        batch_size: int,
        num_workers: int,
    ):
        self.model_size = model_size
        self.dataset_name = dataset_name
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.num_workers = num_workers

        # Validate parameters
        assert model_size in ['CNN_Small', 'CNN_Medium', 'CNN_Large'], f"Model size '{model_size}' not found"
        assert dataset_name in ALL_MODEL_CONFIGS, f"Dataset name '{dataset_name}' not found in ALL_MODEL_CONFIGS"
        assert batch_size > 0, "Batch size must be positive"
        assert epochs > 0, "Number of epochs must be positive"
        assert learning_rate > 0, "Learning rate must be positive"
        assert num_workers >= 0, "Number of workers must be non-negative"


In [11]:
def run_training(training_config):
    """Main function to run the training process."""

    # Set random seed for reproducibility
    torch.manual_seed(DataConfig.SEED)
    np.random.seed(DataConfig.SEED)

    # Get configuration from TrainingConfig
    model_size = training_config.model_size
    dataset_name = training_config.dataset_name
    epochs = training_config.epochs
    learning_rate = training_config.learning_rate
    batch_size = training_config.batch_size
    num_workers = training_config.num_workers

    # Get the ModelConfig for the chosen dataset
    if dataset_name not in ALL_MODEL_CONFIGS:
        print(f"Error: Dataset name '{dataset_name}' not found in ALL_MODEL_CONFIGS.")
        print("Please ensure 'data_preprocessing.py' defines this dataset name.")
        return

    current_dataset_model_config = ALL_MODEL_CONFIGS[dataset_name]
    max_frames_spec = current_dataset_model_config.max_frames_spec

    # Device configuration
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Determine CNN configuration based on model_size
    cnn_configs = {
        "CNN_Small": CNNConfig(
            name="CNN_Small",
            image_size=(DataConfig.N_MELS, current_dataset_model_config.max_frames_spec),
            blocks=[3, 4, 6, 3],
            filters=[64, 128, 256, 512],
            bottleneck=False,
        ),
        "CNN_Medium": CNNConfig(
            name="CNN_Medium",
            image_size=(DataConfig.N_MELS, current_dataset_model_config.max_frames_spec),
            blocks=[3, 4, 6, 3],
            filters=[64, 128, 256, 512],
            bottleneck=True,
        ),
        "CNN_Large": CNNConfig(
            name="CNN_Large",
            image_size=(DataConfig.N_MELS, current_dataset_model_config.max_frames_spec),
            blocks=[3, 6, 20, 3],
            filters=[64, 128, 256, 512],
            bottleneck=True,
        ),
    }

    if model_size not in cnn_configs:
        print(
            f"Error: Invalid model_size '{model_size}'. Choose from {list(cnn_configs.keys())}"
        )
        return

    cnn_config = cnn_configs[model_size]
    print(f"Configuring {cnn_config.name} model...")
    print(
        f"Image size: {cnn_config.image_height}x{cnn_config.image_width}, Blocks: {cnn_config.blocks}, Filters: {cnn_config.filters}"
    )

    # Dataset paths
    base_cache_dir = DataConfig.CACHE_DIR
    model_cache_dir = os.path.join(base_cache_dir, dataset_name)

    print(f"Loading data from: {model_cache_dir}")

    # Datasets
    train_dataset = AudioDataset(
        model_cache_dir, "train", DataConfig.N_MELS, max_frames_spec
    )
    val_dataset = AudioDataset(
        model_cache_dir, "val", DataConfig.N_MELS, max_frames_spec
    )
    test_dataset = AudioDataset(
        model_cache_dir, "test", DataConfig.N_MELS, max_frames_spec
    )

    # DataLoaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=True,
        collate_fn=custom_collate_fn,
    )

    print(f"Train samples: {len(train_dataset)}")
    print(f"Validation samples: {len(val_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    print(f"Using Batch size: {batch_size}")

    # Initialize model, loss, and optimizer
    model = AudioCNN(config=cnn_config)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Run name for W&B
    run_name = f"{model_size}_{dataset_name}_{datetime.now().strftime('%H%M%S')}"

    # Train the model
    trained_model = train_model(
        model,
        train_loader,
        val_loader,
        optimizer,
        criterion,
        device,
        epochs,
        run_name,
        dataset_name,
    )

    # Evaluate on test set
    print(f"\n--- Evaluating {cnn_config.name} on Test Set ({dataset_name}) ---")
    test_loss, test_preds, test_labels, test_probs, test_cm = evaluate_model(
        trained_model, test_loader, criterion, device, return_cm=True
    )

    test_acc = accuracy_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds, average="binary")
    test_roc_auc = roc_auc_score(test_labels, test_probs[:, 1])

    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test F1-score: {test_f1:.4f}")
    print(f"Test ROC AUC: {test_roc_auc:.4f}")

    # Plot and save confusion matrix
    cm_plot_path = plot_confusion_matrix(test_cm, run_name=run_name, save_dir="results")

    # Log test metrics to W&B
    wandb.log(
        {
            "test_loss": test_loss,
            "test_accuracy": test_acc,
            "test_f1_score": test_f1,
            "test_roc_auc": test_roc_auc,
            "confusion_matrix": wandb.Image(cm_plot_path),
        }
    )
    wandb.finish()

    return trained_model


In [ ]:
# Configure training parameters
training_params = {
    "model_size": "CNN_Small",
    "dataset_name": "cnn_balanced_dataset",
    "epochs": 20,
    "learning_rate": 1e-4,
    "batch_size": 32,
    "num_workers": 8,
}

# Initialize TrainingConfig with the specified parameters
training_config = TrainingConfig(**training_params)

# Print training configuration
print("=== CNN Training Configuration ===")
print(f"Model Size: {training_config.model_size}")
print(f"Dataset: {training_config.dataset_name}")
print(f"Epochs: {training_config.epochs}")
print(f"Learning Rate: {training_config.learning_rate}")
print(f"Batch Size: {training_config.batch_size}")
print(f"Num Workers: {training_config.num_workers}")
print("==================================")

# Run training with the configured TrainingConfig
trained_model = run_training(training_config)


=== CNN Training Configuration ===
Model Size: CNN_Small
Dataset: cnn_balanced_dataset
Epochs: 20
Learning Rate: 0.0001
Batch Size: 32
Num Workers: 8
Using device: cuda
Configuring CNN_Small model...
Image size: 128x250, Blocks: [3, 4, 6, 3], Filters: [64, 128, 256, 512]
Loading data from: F:\Deepfake-Audio-Detector\processed_dataset\cnn_balanced_dataset
Train samples: 61022
Validation samples: 13205
Test samples: 13650
Using Batch size: 32


Starting training for 20 epochs on cuda...


Epoch 1/20 [Train]:   0%|          | 0/1907 [00:00<?, ?it/s]